In [1]:
import numpy as np
from skimage.io import imread
import matplotlib.pyplot as plt
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk, ImageEnhance
import btrack
import pandas as pd
import seaborn as sns
import os 
import json

In [2]:
WORKING_DIR = 'd:\\David\\endoderm_migration\\2024_09_09_FoxA2_tdTom_c211to300_BmpAct_day2To3'
BRIGHTFIELD_IMAGES_FOLDER = WORKING_DIR+'\\brightfield'

In [3]:
# read the brightfield images and display them in one big figure (title is the filename)
def read_brightfield_images(PATH):
    brightfield_images = dict()
    # read all the images in the folder
    for filename in os.listdir(PATH):
        if filename.endswith(".tif"):
            img = imread(PATH+'\\'+filename)
            brightfield_images[filename] = img # store the image in a dictionary with the filename as key

    # display the images
    fig, ax = plt.subplots(1, len(brightfield_images), figsize=(20, 20))
    for i, (filename, img) in enumerate(brightfield_images.items()):
        # display the image without axis
        ax[i].imshow(img, cmap='gray')
        ax[i].axis('off')
        ax[i].set_title(filename, fontsize=5)
    return brightfield_images

In [4]:
# Global dictionary to store mask parameters
masks_parameters_json = {}

class ImageGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("TIF Image Loader and Mask Fitter")

        self.canvas = tk.Canvas(root)
        self.canvas.pack(expand=tk.YES, fill=tk.BOTH)

        self.load_folder_button = tk.Button(root, text="Load Folder", command=self.load_folder)
        self.load_folder_button.pack()

        self.next_button = tk.Button(root, text="Next Image", command=self.next_image)
        self.next_button.pack()

        self.prev_button = tk.Button(root, text="Previous Image", command=self.prev_image)
        self.prev_button.pack()

        self.update_position_button = tk.Button(root, text="Update Position", command=self.update_position)
        self.update_position_button.pack()

        self.apply_mask_button = tk.Button(root, text="Apply Mask", command=self.apply_mask)
        self.apply_mask_button.pack()

        self.width_label = tk.Label(root, text="Width:")
        self.width_label.pack()
        self.width_entry = tk.Entry(root)
        self.width_entry.pack()
        self.width_entry.insert(0, "1000")

        self.height_label = tk.Label(root, text="Height:")
        self.height_label.pack()
        self.height_entry = tk.Entry(root)
        self.height_entry.pack()
        self.height_entry.insert(0, "2700")

        self.tilt_label = tk.Label(root, text="Tilt Angle (degrees):")
        self.tilt_label.pack()
        self.tilt_entry = tk.Entry(root)
        self.tilt_entry.pack()
        self.tilt_entry.insert(0, "3")

        self.cursor_position = None
        self.image = None
        self.photo_image = None
        self.image_id = None
        self.original_width = None
        self.original_height = None
        self.scaling_factor_x = 1
        self.scaling_factor_y = 1

        self.image_files = []
        self.current_image_index = -1

        self.canvas.bind("<Button-1>", self.update_cursor_position)

    def load_folder(self):
        folder_path = filedialog.askdirectory()
        if folder_path:
            self.image_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith(".tif")]
            if self.image_files:
                self.current_image_index = 0
                self.load_image()

    def next_image(self):
        if self.current_image_index < len(self.image_files) - 1:
            self.current_image_index += 1
            self.load_image()

    def prev_image(self):
        if self.current_image_index > 0:
            self.current_image_index -= 1
            self.load_image()

    def load_image(self):
        if self.image_files:
            file_path = self.image_files[self.current_image_index]
            self.image = Image.open(file_path)
            self.original_width, self.original_height = self.image.size

            # Resize the image
            max_width = 800  # Max width you want
            max_height = 600  # Max height you want
            self.image.thumbnail((max_width, max_height), Image.LANCZOS)
            resized_width, resized_height = self.image.size

            # Calculate scaling factors
            self.scaling_factor_x = self.original_width / resized_width
            self.scaling_factor_y = self.original_height / resized_height

            self.photo_image = ImageTk.PhotoImage(self.image)

            if self.image_id:
                self.canvas.delete(self.image_id)

            self.image_id = self.canvas.create_image(0, 0, anchor=tk.NW, image=self.photo_image)
            self.canvas.config(scrollregion=self.canvas.bbox(tk.ALL))

            self.root.title(f"TIF Image Loader and Mask Fitter - {os.path.basename(file_path)}")

            # Load existing cursor position if available
            image_file = os.path.basename(file_path)
            if image_file in masks_parameters_json and 'center_x' in masks_parameters_json[image_file] and 'center_y' in masks_parameters_json[image_file]:
                center_x = masks_parameters_json[image_file]['center_x'] / self.scaling_factor_x
                center_y = masks_parameters_json[image_file]['center_y'] / self.scaling_factor_y
                self.draw_cursor(center_x, center_y)

    def update_cursor_position(self, event):
        # Adjust cursor position to match original image coordinates
        self.cursor_position = (event.x * self.scaling_factor_x, event.y * self.scaling_factor_y)
        self.draw_cursor(event.x, event.y)

    def draw_cursor(self, display_x, display_y):
        self.canvas.delete("cursor")
        if self.cursor_position:
            self.canvas.create_line(display_x-70, display_y, display_x+70, display_y, fill="red", tags="cursor")
            self.canvas.create_line(display_x, display_y-250, display_x, display_y+250, fill="red", tags="cursor")

    def update_position(self):
        if self.cursor_position:
            image_file = os.path.basename(self.image_files[self.current_image_index])
            if image_file not in masks_parameters_json:
                masks_parameters_json[image_file] = {}
            masks_parameters_json[image_file].update({
                'center_x': self.cursor_position[0],
                'center_y': self.cursor_position[1],
                'width': None,
                'height': None,
                'tilt_angle': None,
            })
            print(f"Cursor position updated for {image_file}: {self.cursor_position}")

    def apply_mask(self):
        if not self.cursor_position:
            print("No cursor position selected.")
            return
        
        try:
            width = float(self.width_entry.get())
            height = float(self.height_entry.get())
            tilt_angle = float(self.tilt_entry.get())
        except ValueError:
            print("Invalid input for width, height, or tilt angle.")
            return

        # Convert tilt angle to radians
        tilt_angle_rad = np.radians(tilt_angle)

        # Create a meshgrid
        x = np.arange(self.original_width)
        y = np.arange(self.original_height)
        X, Y = np.meshgrid(x, y)

        # Center coordinates
        center_x = self.cursor_position[0]
        center_y = self.cursor_position[1]

        # Translate coordinates to the center
        X_translated = X - center_x
        Y_translated = Y - center_y

        # Rotate coordinates
        X_rotated = X_translated * np.cos(tilt_angle_rad) + Y_translated * np.sin(tilt_angle_rad)
        Y_rotated = -X_translated * np.sin(tilt_angle_rad) + Y_translated * np.cos(tilt_angle_rad)

        # Create a mask for the rectangle
        rectangle_mask = (X_rotated > -width/2) & (X_rotated < width/2) & (Y_rotated > -height/2) & (Y_rotated < height/2)

        # Create masks for the half circles
        circle_mask1 = (X_rotated)**2 + (Y_rotated - height/2)**2 < (width/2)**2
        circle_mask2 = (X_rotated)**2 + (Y_rotated + height/2)**2 < (width/2)**2

        # Combine the masks
        mask = rectangle_mask | circle_mask1 | circle_mask2

        # Save mask coordinates
        mask_coordinates = np.argwhere(mask)

        # Create an RGBA mask image with alpha transparency
        mask_image = Image.new("RGBA", (self.original_width, self.original_height), (255, 0, 0, 0))
        mask_image_data = mask_image.load()
        for coord in mask_coordinates:
            mask_image_data[coord[1], coord[0]] = (255, 0, 0, 51)  # 51 is for 20% transparency

        # Convert the original image to RGBA and resize to original dimensions
        overlay_image = self.image.convert("RGBA")
        overlay_image = overlay_image.resize((self.original_width, self.original_height), Image.LANCZOS)

        # Composite the overlay image and the mask image
        blended_image = Image.alpha_composite(overlay_image, mask_image)

        # Resize the blended image to match the canvas size
        blended_image.thumbnail((self.canvas.winfo_width(), self.canvas.winfo_height()), Image.LANCZOS)
        self.photo_image = ImageTk.PhotoImage(blended_image)
        
        if self.image_id:
            self.canvas.delete(self.image_id)
        
        self.image_id = self.canvas.create_image(0, 0, anchor=tk.NW, image=self.photo_image)
        self.canvas.config(scrollregion=self.canvas.bbox(tk.ALL))

        print("Mask applied and displayed on the image.")
        print("Mask coordinates saved.")

        # Update the saved data with mask details
        image_file = os.path.basename(self.image_files[self.current_image_index])
        if image_file not in masks_parameters_json:
            masks_parameters_json[image_file] = {}
        masks_parameters_json[image_file].update({
            'width': width,
            'height': height,
            'tilt_angle': tilt_angle,
        })

if __name__ == "__main__":
    root = tk.Tk()
    gui = ImageGUI(root)
    root.mainloop()

    time = str(pd.Timestamp.now()).replace(' ', '_').replace(':', '-').replace('.', '-')
    # save masks_parameters_json file
    with open(os.path.join(WORKING_DIR, 'masks_parameters_'+time+'.json'), 'w') as f:
        json.dump(masks_parameters_json, f)

Mask applied and displayed on the image.
Mask coordinates saved.
Cursor position updated for BRIGHTFIELD_01.tif: (1002.6666666666666, 1868.615)
Cursor position updated for BRIGHTFIELD_02.tif: (988.4444444444443, 1882.825)
Mask applied and displayed on the image.
Mask coordinates saved.
Cursor position updated for BRIGHTFIELD_03.tif: (1024.0, 2053.3450000000003)
Mask applied and displayed on the image.
Mask coordinates saved.
Cursor position updated for BRIGHTFIELD_04.tif: (1009.7777777777777, 1854.4050000000002)
Mask applied and displayed on the image.
Mask coordinates saved.
Cursor position updated for BRIGHTFIELD_05.tif: (1052.4444444444443, 1875.72)
Cursor position updated for BRIGHTFIELD_05.tif: (1052.4444444444443, 1875.72)
Mask applied and displayed on the image.
Mask coordinates saved.
Mask applied and displayed on the image.
Mask coordinates saved.
Cursor position updated for BRIGHTFIELD_06.tif: (1002.6666666666666, 2046.2400000000002)
Mask applied and displayed on the image.
M

In [5]:
masks_parameters_json

{'BRIGHTFIELD_01.tif': {'width': None,
  'height': None,
  'tilt_angle': None,
  'center_x': 1002.6666666666666,
  'center_y': 1868.615},
 'BRIGHTFIELD_02.tif': {'center_x': 988.4444444444443,
  'center_y': 1882.825,
  'width': 1000.0,
  'height': 2700.0,
  'tilt_angle': -1.0},
 'BRIGHTFIELD_03.tif': {'center_x': 1024.0,
  'center_y': 2053.3450000000003,
  'width': 1000.0,
  'height': 2700.0,
  'tilt_angle': -1.0},
 'BRIGHTFIELD_04.tif': {'center_x': 1009.7777777777777,
  'center_y': 1854.4050000000002,
  'width': 1000.0,
  'height': 2700.0,
  'tilt_angle': -1.0},
 'BRIGHTFIELD_05.tif': {'center_x': 1052.4444444444443,
  'center_y': 1875.72,
  'width': 1000.0,
  'height': 2600.0,
  'tilt_angle': -1.0},
 'BRIGHTFIELD_06.tif': {'center_x': 1002.6666666666666,
  'center_y': 2046.2400000000002,
  'width': 950.0,
  'height': 2600.0,
  'tilt_angle': -1.0},
 'BRIGHTFIELD_07.tif': {'center_x': 1016.8888888888888,
  'center_y': 1882.825,
  'width': 950.0,
  'height': 2600.0,
  'tilt_angle': -1.